In [9]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import json
import os
import mayavi
from mayavi import mlab
mlab.init_notebook('x3d', 600, 600) # x3d or png

Notebook initialized with x3d backend.


# Notes
Translation tests measure the most basic feature of an interface tracking method. In this test, a hollow square, two tilted hollow square and a hollow circle are advected in constant velocity fields, periodic boudnary conditions are set at each of the . he material area and the shape should remain unchanged through the translation process.

## Model setup

The initial setup is

| Parameter  | value                         | 
|------------|-------------------------------|
| domain     | [0,2] $\times$ [0,2]          | 
| grids      | 100 $\times$ 100 $\times$ 100       |
| $h$        | 0.02                                |
| shape      | D = 0.4, d = 0.2                    |
| angle      | $\theta_1$ = 26.57, $\theta_2$ = 45 |
| velocity   | $\vec u = [1,1,1]^T$          |
| Time step  | $k=0.005$         |
| $n_{step}$ | $400$         |

# Setup

## Generate input file

In [10]:
f=h5py.File('input.h5','w') # open hdf5 file

OSError: Unable to create file (unable to truncate a file which is already open)

In [11]:
# Initialize array
nx = 100
ny = 100
nz = 100
X,Y,Z = np.meshgrid(np.arange(ny),np.arange(nz),np.arange(nx))
phi = np.zeros([nz,ny,nx])
cx = np.zeros([nz,ny,nx])
cy = np.zeros([nz,ny,nx])
cz = np.zeros([nz,ny,nx])
u = np.zeros([nz,ny,nx])
v = np.zeros([nz,ny,nx]) 
w = np.zeros([nz,ny,nx]) 

In [12]:
# hollow cube 1
phi[5:45,5:45,5:45] = 1.0
phi[15:35,15:35,15:35] = 0.0
cx[5:45,5:45,5:45] = 0.5
cx[15:35,15:35,15:35] = 0.0
cy[5:45,5:45,5:45] = 0.5
cy[15:35,15:35,15:35] = 0.0
cz[5:45,5:45,5:45] = 0.5
cz[15:35,15:35,15:35] = 0.0
phi[5:45,5:45,5:45] = 1.0
phi[15:35,15:35,15:35] = 0.0

# hollow cube 2
phi[5:45,5:45,5:45] = 1.0
phi[15:35,15:35,15:35] = 0.0
cx[5:45,5:45,5:45] = 0.5
cx[15:35,15:35,15:35] = 0.0
cy[5:45,5:45,5:45] = 0.5
cy[15:35,15:35,15:35] = 0.0
cz[5:45,5:45,5:45] = 0.5
cz[15:35,15:35,15:35] = 0.0
phi[5:45,5:45,5:45] = 1.0
phi[15:35,15:35,15:35] = 0.0

lvl = np.zeros([20,20,20])
lvl_lo = np.zeros([20,20,20])
lvl_hi = np.zeros([20,20,20])
es = np.zeros([20,20,20])
x = np.arange(20) / 20 + 0.5/20
x_lo = np.arange(20) / 20 
x_hi = np.arange(20) / 20 + 1/20

# hollow sphere 
for i in range(20):
    for j in range(20):
        for k in range(20):
            sph1 = np.sqrt(x[i]**2+x[j]**2+x[k]**2) 
            dis1 = 1.0 - sph1
            dis2 = sph1 - 0.5
            lvl[i,j,k] = min(dis1,dis2)
            sph2 = np.sqrt(x_lo[i]**2+x_lo[j]**2+x_lo[k]**2) 
            dis1 = 1.0 - sph2
            dis2 = sph2 - 0.5
            lvl_lo[i,j,k] = min(dis1,dis2)
            sph3 = np.sqrt(x_hi[i]**2+x_hi[j]**2+x_hi[k]**2) 
            dis1 = 1.0 - sph3
            dis2 = sph2 - 0.5
            lvl_hi[i,j,k] = min(dis1,dis2)
for i in range(20):
    for j in range(20):
        for k in range(20):
            if lvl[i,j,k]>=0 and lvl_lo[i,j,k]>=0 and lvl_hi[i,j,k]>=0:
                es[i,j,k] = 1.0
            elif lvl[i,j,k]<=0 and lvl_lo[i,j,k]<=0 and lvl_hi[i,j,k]<=0:
                es[i,j,k] = 0.0
            else:
                for i in range(1000):
                    for j in range(1000):
                        for k in range(20):
            
mlab.clf()
mlab.contour3d(es,contours=8,opacity=0.2)
mlab.outline(extent=[0,nx,0,ny,0,nz])
mlab.colorbar()

IndentationError: expected an indented block (<ipython-input-12-f404f83a7ce4>, line 61)

In [52]:
u = np.zeros([nz,ny,nx])+ 1.0
v = np.zeros([nz,ny,nx]) + 1.0
w = np.zeros([nz,ny,nx]) + 1.0

In [53]:
f.create_group('phi')
grp = f['phi']
grp.create_dataset("init", data=phi)
f.create_group('u')
grp = f['u']
grp.create_dataset("init", data=u)
f.create_group('v')
grp = f['v']
grp.create_dataset("init", data=v)
f.create_group('w')
grp = f['w']
grp.create_dataset("init", data=w)
f.create_group('cx')
grp = f['cx']
grp.create_dataset("init", data=cx)
f.create_group('cy')
grp = f['cy']
grp.create_dataset("init", data=cy)
f.create_group('cz')
grp = f['cz']
grp.create_dataset("init", data=cz)

<HDF5 dataset "init": shape (100, 100, 100), type "<f8">

In [54]:
f.close()